In [2]:
# Create a connection

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

In [3]:
# Create a keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS killrvideo
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [4]:
# Set the keyspace
try:
    session.set_keyspace('killrvideo')
except Exception as e:
    print(e)

In [69]:
query = """
CREATE TABLE IF NOT EXISTS videos(
    video_id timeuuid,
    added_date timestamp,
    description text,
    title text,
    user_id uuid,
    PRIMARY KEY(video_id)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

- Import data into videos table in killrvideo keyspace

In [70]:
import csv
import datetime
import uuid

In [72]:
file = r'C:\Users\abdul\cassandra-fundamentals\foundations-of-apache-cassandra\data\videos.csv'

with open(file, encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        query = "INSERT INTO videos(video_id, added_date, description, title, user_id)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        video_id = uuid.UUID(line[0])
        added_date = line[1]
        if "/" in added_date:
            added_date = datetime.datetime.strptime(added_date, "%m/%d/%Y").date()
        else:
            added_date = datetime.datetime.strptime(added_date, "%Y-%m-%d %H:%M:%S%z") #2015-03-29 08:00:03+0000
            
        description = line[2]
        title = line[3]
        user_id = uuid.UUID(line[4])
        session.execute(query, (video_id, added_date, description, title, user_id)) 

In [24]:
# with open(file) as f:
#     reader = csv.reader(f)
#     next(reader)
#     for line in reader:
#         print(line)

In [63]:
# file = r'C:\Users\abdul\cassandra-fundamentals\foundations-of-apache-cassandra\data\videos.csv'

# with open(file, encoding="utf8") as f:
#     reader = csv.reader(f)
#     next(reader)
#     for line in reader:
#         query = "INSERT INTO videos(video_id, added_date, description, title, user_id)"
#         query = query + "VALUES(%s, %s, %s, %s, %s)"
        
#         added_date = uuid.UUID(line[0])
#         user_id = uuid.UUID(line[4])
#         session.execute(query, (added_date, line[1], line[2], line[3], user_id))

In [73]:
query = "SELECT * FROM videos LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.video_id, row.added_date, row.description, row.title, row.user_id)

26461a70-14bd-11e5-ad08-8438355b7e3a 2014-05-06 23:00:00 At Comcast we are working on the future of television. Change and innovation are happening more rapidly than ever thanks to the cloud based X1 platform ... Webinar: Building Blocks for the Future of Television 10d5c76c-8767-4db3-8050-e19e015b524c
2645e79c-14bd-11e5-a456-8438355b7e3a 2011-10-20 23:00:00 DataStax is the developer of DataStax Enterprise, a distributed, scalable, and highly available database platform that delivers optimal performance either on ... DataStax Cassandra Tutorials - Understanding partitioning and replication in Cassandra 10d5c76c-8767-4db3-8050-e19e015b524c
9056808b-ca65-1bfb-9957-3bea148dfdce 2015-03-09 00:00:02 New hire Chip (Chris Hemsworth) learns it's not easy working alongside Lucious (Kenan Thompson), Cookie (Sasheer Zamata), Jamal (Jay Pharoah), Hakeem ... Empire Promo - SNL 220077ff-be79-4f20-8603-1b9c97dbafa6
264601a3-14bd-11e5-8c2e-8438355b7e3a 2011-12-29 23:00:00 Tyler Hobbs - Flexibility: Py

In [74]:
query = "SELECT COUNT(video_id) FROM videos"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(system_count_video_id=430)


In [76]:
query = "SELECT * FROM videos WHERE video_id = 352f3b6c-38d7-1c8d-b222-623ba1f44ec1 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.added_date)

2015-03-29 08:00:03


### Testing added_date inserted

In [77]:
query = "SELECT * FROM videos WHERE video_id = 352f3b6c-38d7-1c8d-b222-623ba1f44ec1 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.added_date)

2015-03-29 08:00:03


In [79]:
query = "SELECT * FROM videos WHERE video_id = 2645e8d1-14bd-11e5-958e-8438355b7e3a "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.added_date)

2011-10-20 23:00:00


## Drop table

In [80]:
query = "DROP TABLE IF EXISTS videos"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [81]:
session.shutdown()
cluster.shutdown()

In [65]:
# import datetime

In [66]:
# datetime.datetime.strptime("10/5/2011", "%m/%d/%Y")

In [67]:
# datetime.datetime.strptime("2015-03-29 08:00:03+0000", "%Y-%m-%d %H:%M:%S%z")